In [1]:
import os
import scenic
# scenic_script = "./examples/carla/Carla_Challenge/carlaChallenge5.scenic"
scenic_script = "./examples/carla/car.scenic"
scenario = scenic.scenarioFromFile(scenic_script)

# from scenic.domains.driving.roads import ManeuverType, Network
# map_path = '/Users/edwardkim/Desktop/Scenic-devel/examples/carla/../../tests/formats/opendrive/maps/CARLA/Town05.xodr'
# network = Network.fromFile(map_path)

/Users/edwardkim/Desktop/Scenic_Query/Scenic/src/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')
/Users/edwardkim/Desktop/Scenic_Query/Scenic/src/scenic/simulators/carla/model.scenic:56: UserWarning: the "carla" package is not installed; will not be able to run dynamic simulations
  warnings.warn('the "carla" package is not installed; '


In [2]:
from scenic.core.vectors import OrientedVector, Vector
import shapely.geometry
import matplotlib.pyplot as plt
from scenic.core.type_support import toVector
from scenic.core.object_types import Point
from scenic.core.type_support import TypeChecker

type(scenario.egoObject.position)


scenic.core.regions.PointInRegionDistribution

In [3]:
# creating a ego's visible sector region to find the lane's triangles is too much optimization, 
# and can violate what is actually written in the scenic script
from scenic.core.distributions import *
from scenic.core.vectors import Vector
from scenic.core.regions import SectorRegion
import math

ego_visibleDistance = 100
ego_viewAngle = 135 #deg
ego_labelled_position = Vector(0, 0)
ego_labelled_heading = 0 #deg
egoVisibleRegion = SectorRegion(ego_labelled_position, ego_visibleDistance, \
                                math.radians(ego_labelled_heading), math.radians(ego_viewAngle))

# step 1: get rid of this optimization
ego = scenario.egoObject
smt_file_path = './test_smt_encoding.smt2'
open(smt_file_path, 'w').close()
writeSMTtoFile(smt_file_path, '(set-logic QF_NRA)')

cached_variables = {}
cached_variables['variables'] = []
cached_variables['current_obj_pos'] = (27.939338146883905, 87.14617192114929)
cached_variables['egoVisibleRegion'] = egoVisibleRegion.polygon

ego_pos_smt_var = ego.position.encodeToSMT(smt_file_path, cached_variables, debug = True)


# step 2: create the testing pipeline above

# step 3: add smt translation

# step 4: testing with Scenic scripts and their samples

RuntimeParseError: cannot take the len of a random value